# نمونه ساده AutoGen

در این نمونه کد، شما از چارچوب هوش مصنوعی [AutoGen](https://aka.ms/ai-agents/autogen) برای ایجاد یک عامل پایه استفاده خواهید کرد.

هدف این نمونه این است که مراحل مورد نیاز را به شما نشان دهد که در نمونه‌های کد اضافی بعدی هنگام پیاده‌سازی الگوهای مختلف عامل‌محور استفاده خواهیم کرد.


## وارد کردن بسته‌های مورد نیاز پایتون


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## ایجاد کلاینت

در این نمونه، از [مدل‌های GitHub](https://aka.ms/ai-agents-beginners/github-models) برای دسترسی به LLM استفاده خواهیم کرد.

`model` به صورت `gpt-4o-mini` تعریف شده است. مدل را به یکی دیگر از مدل‌های موجود در بازار مدل‌های GitHub تغییر دهید تا نتایج مختلف را مشاهده کنید.

به عنوان یک آزمایش سریع، فقط یک درخواست ساده اجرا می‌کنیم - `پایتخت فرانسه چیست؟`


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## تعریف عامل

حالا که `client` را تنظیم کرده‌ایم و تأیید کرده‌ایم که به درستی کار می‌کند، بیایید یک `AssistantAgent` ایجاد کنیم. هر عامل می‌تواند دارای موارد زیر باشد:  
**name** - یک نام کوتاه که در جریان‌های چندعاملی برای ارجاع به آن مفید خواهد بود.  
**model_client** - کلاینتی که در مرحله قبلی ایجاد کردید.  
**tools** - ابزارهایی که عامل می‌تواند برای انجام یک وظیفه از آن‌ها استفاده کند.  
**system_message** - پیام سیستمی که وظیفه، رفتار و لحن LLM را تعریف می‌کند.  

شما می‌توانید پیام سیستمی را تغییر دهید تا ببینید LLM چگونه پاسخ می‌دهد. ما در درس شماره ۴ به موضوع `tools` خواهیم پرداخت.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## اجرای عامل

تابع زیر عامل را اجرا می‌کند. ما از متد `on_message` برای به‌روزرسانی وضعیت عامل با پیام جدید استفاده می‌کنیم.

در این مثال، وضعیت را با یک پیام جدید از کاربر که عبارت است از `"برای من یک تعطیلات آفتابی عالی برنامه‌ریزی کن"` به‌روزرسانی می‌کنیم.

می‌توانید محتوای پیام را تغییر دهید تا ببینید مدل زبانی بزرگ (LLM) چگونه به شکل متفاوتی پاسخ می‌دهد.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
